In [2]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [13]:

X = mnist['data']
X.shape
y = mnist['target'].ravel()
X_w_bias = np.c_[np.ones([len(X), 1]), X]


In [14]:
#Normalize X
X_w_bias = X_w_bias /255.0 #To the X go from 0 to 1, instead of 0 to 255
X_train, X_test, y_train, y_test = train_test_split(X_w_bias, y, test_size = 0.4, random_state = 2)


In [27]:
#Let's take a validation set 
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 2)
n_inputs = X_train.shape[1] #Number of features (all the pixels plus the bias term) 
n_outputs = len(np.unique(y_train)) #Number of possible classes (0 to 9 )
y_train

array(['1', '0', '5', ..., '1', '8', '3'], dtype=object)

In [95]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
#One Hot the classes with a OneHotEncoder in order to have de probabilities of being any of this chances 
#on the labels 
#It is important to keep our encoded data as arrays
y_train_encoded = encoder.fit_transform(y_train.reshape(-1,1)).toarray()
y_val_encoded = encoder.fit_transform(y_val.reshape(-1,1)).toarray()
y_test_encoded = encoder.fit_transform(y_test.reshape(-1,1)).toarray()


In [99]:
y_train_encoded.shape

(17203, 10)

In [100]:
def Softmax(logits):
  exps = np.exp(logits)
  exp_sums = np.sum(exps, axis=1, keepdims=True)
  return exps / exp_sums



In [141]:

#Lets do the Batch Gradient descent without the softmaxt algorithm
eta = 0.1 #Learning rate
n_iterations = 2501
m = len(X_train)
epsilon = 1e-7
thetas = np.random.rand(n_inputs, n_outputs) #Thetas for all the features and all the possible classes (all pixels plus bias X 0 to 9 classes)

for iteration in range(n_iterations):
  logits = X_train.dot(thetas)
  y_proba = Softmax(logits)
  loss = -np.mean(np.sum(y_train_encoded * np.log(y_proba + epsilon), axis=1)) #Cross entropy cost function
  if iteration % 500 == 0:
    print(iteration, loss)
  erro = y_proba - y_train_encoded
  gradients = 1/m * X_train.T.dot(erro)
  thetas = thetas - eta * gradients

  

0 5.062183557169024
500 0.4772517929622487
1000 0.39699479388965025
1500 0.3617676820405957
2000 0.34050550170449406
2500 0.32577036403268544


In [142]:
thetas


array([[0.23867834, 0.52739878, 0.80025867, ..., 0.77965808, 0.10116605,
        0.10966146],
       [0.90179001, 0.39320626, 0.67858522, ..., 0.78666287, 0.18191937,
        0.61427201],
       [0.31336247, 0.94377811, 0.74271074, ..., 0.86125228, 0.63717742,
        0.62499266],
       ...,
       [0.62450795, 0.13821028, 0.89058998, ..., 0.71429873, 0.69670821,
        0.56683896],
       [0.81314645, 0.07637499, 0.2224422 , ..., 0.79962577, 0.90421759,
        0.67183655],
       [0.52307219, 0.01025581, 0.82452387, ..., 0.06324123, 0.49287149,
        0.41399952]])

In [148]:
#Let's do predictions for the validation set and check the accuracy score
logits = X_val.dot(thetas)
y_proba = Softmax(logits)
prediction = np.argmax(y_proba, axis=1) #Returns the INDEX with the highest probability
y_val_int = [int(string_val) for string_val in y_val ]
score_accuracy = np.mean(prediction == y_val_int)
prediction


array([0, 9, 5, ..., 4, 8, 0])

let's add a bit of $\ell_2$ regularization 

In [150]:
#Lets do the Batch Gradient descent without the softmaxt algorithm
eta = 0.1 #Learning rate
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7
alpha = 0.1 #Regularization Hyperparameter



thetas = np.random.rand(n_inputs, n_outputs) #Thetas for all the features and all the possible classes (all pixels plus bias X 0 to 9 classes)

for iteration in range(n_iterations):
  logits = X_train.dot(thetas)
  y_proba = Softmax(logits)
  X_entropy_loss = -np.mean(np.sum(y_train_encoded * np.log(y_proba + epsilon), axis=1)) #Cross entropy cost function
  l2_loss = 1/2 * np.sum(np.square(thetas[1:]))
  loss = X_entropy_loss + alpha * l2_loss

  if iteration % 500 == 0:
    print(iteration, loss)
  erro = y_proba - y_train_encoded
  gradients = 1/m * X_train.T.dot(erro) + np.r_[np.zeros([1,n_outputs]), alpha * thetas[1:]]
  thetas = thetas - eta * gradients


0 135.1887482273817
500 1.1065121409072043
1000 1.1009987297865185
1500 1.1009982548262804
2000 1.1009980150644496
2500 1.1009977753154552
3000 1.1009975355692303
3500 1.1009972958257743
4000 1.1009970560850877
4500 1.10099681634717
5000 1.1009965766120213


In [151]:
logits = X_val.dot(thetas)
y_proba = Softmax(logits)
prediction = np.argmax(y_proba, axis=1) #Returns the INDEX with the highest probability
y_val_int = [int(string_val) for string_val in y_val ]
score_accuracy = np.mean(prediction == y_val_int)
score_accuracy


0.8598000465008138